<h1 style="color:#333333; text-align:center; line-height: 0;">Reinforcement Learning | Assignment 1</h1>

<br/><br/>

This notebook covers two classical approaches in RL: **value iteration** and **policy iteration**.

Before you start, please install OpenAI gym for Python [here](https://gym.openai.com/docs/).

Complete the code snippets given in the Section 3: there are 4 places to insert your code, one string field to enter the algorithm name and string fields for your first and last name. The latter are needed to automatically save the results of the algorithms deployment in .json file. After you did that, please upload the notebook (.ipynb) and .json via https://forms.gle/yJepv3CzfwMRnQULA.

* Problem 1.1 - Value Iteration (15 points)
* Problem 1.2 - Policy Iteration (15 points)
* Problem 1.3 - FrozenLake8x8 (2 points)

***

<h2 style="color:#A7BD3F;">Section 1: Theory recap</h2>

$x$ - state from state space $\mathbb{X}$

$u$ - action from action space $\mathbb{U}$

$R$ - reward function as a random variable ($\rho$ as a deterministic function)

$\kappa$ - policy (in deterministic case)

$\vartheta$ - policy parameters from parameter space $\Pi$

Discrete time deterministic system (DT):

$x_{k+1} = f(x_k, u_k)$

$u_k = \kappa(x_k)$

The core problem that is being solved in RL is teaching the agent to behave *optimally* in a specific environment.

The numerical definition of what is *optimal* is given by the **objective function** $J$, which is typically maximized in the context of MDPs (Markov Decision Processes):

$X_{k+1} \sim P_{X}\left(x_{k+1} \mid x_{u}, u_{n}\right)$

$U_{k} \sim P_{U}^{\vartheta}\left(u_{k} \mid x_{k}\right)$

The next state and next control are sampled from the respective probability distributions. $P_{X}, P_{U}^{\vartheta}$ are probability distribution (or density if $\mathbb{X}$, $\mathbb{U}$ are continuous) functions (PDF). Policy is represented here by a PDF with parameters $\vartheta$, but may be a definite function (Markov policy) $ = \kappa(X_{k})$

The goal then is to teach the agent to maximize the *total expected reward* it earns over some time horizon (theoretically, it is an infinite time horizon) by selecting the best action as dictated by the value function. The agent learns to maximize rewards as it transitions from state to state, taking actions in each state.

Consider the following $\infty$-horizon optimal control problem for an MDP:
$$
J(x) = \max _{\vartheta} \mathbb{E}\left[\sum_{k=0}^{\infty} \gamma^{k} \rho\left(X_{k}, \kappa\left(X_{k}\right)\right) \mid X_{0}=x\right]
$$

The value function is defined in the following way:

$V(x) = \underset{\vartheta}{\max} \ J^{\vartheta}(x)$

Let us use the denotation $X_{+}^{u} \sim P_{X}(x \mid x, u)$ for the next state.

***

<h2 style="color:#A7BD3F;">Section 2: OpenAI FrozenLake environment</h2>

For this homework we will be exploring agent training in the grid world environment called *FrozenLake*. Read more about it [here](https://gym.openai.com/envs/FrozenLake-v0/). To summarize:
* FrozenLake is a 2D grid world
* There are 2 variants of the environment: 4x4 and 8x8. We will try both.
* There are 4 types of grid cells (S, F, H, G). *S* is the starting point, *G* is the goal, *F* is a frozen surface and *H* is a hole.
* If an agent steps onto a slippery surface, it may slip and not end up in the next desired state for which it took an action (think, transition probabilities!).
* The rewards are sparse: the agent receives a reward of 1 when reaching the goal and 0 otherwise. If the agent falls into a hole, the episode is over.
* Note: we will not be using the slippery version of the environment for simplicity.

Familiarize yourself with the code below and run it.

In [ ]:
import gym
import numpy as np
import collections
import sys
from tqdm import tqdm
from IPython.display import clear_output
import time

In [ ]:
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print(observation)
        if done:
            #print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

***

<h2 style="color:#A7BD3F;">Section 3 - Dynamic Programming algorithms for MDPs</h2>

The pseudocode for the general case of Value Iteration and Policy Iteration is given below.

For the environments with discrete state and action spaces, such as FrozenLake, it becomes shorter and easier to comprehend, see pp. 74-83 in Chapter 4 of the class text <sup>[3]</sup>. Be aware of the differences in the notation: $s$ instead of $x$, etc.

We have implemented a class `MDP`, containing methods that are common to these algorithms.

**\_model_transits_rewards** method implements experience gaining, i.e. performing random actions in order to make $\infty$-horizon reward prediction possible.

The resultant policies converge to the optimal policy in terms of the Functional norm, meaning that $\lim \limits_{k \rightarrow \infty} \|V_k - \hat{V}\| = \lim \limits_{k \rightarrow \infty} \sup \limits_{x \in \mathbb{X}} \|V_k(x) - \hat{V}(x)\| = 0$. However, real-world implementation requires practical termination criteria. Here we stop the procedure when the maximal change of the policy among all the states during one optimization loop does not exceed a certain constant $\nu$, see pseudocode for details. The value $\nu$ is a tuning parameter for both algorithms.

The initial values for Value Iteration could be set to $0$. Same for the Policy Iteration, but only for the finite state spaces: in continuous case the criteria for the feasible initialization are discussed later in the course.

<img src="algorithms.png" alt="Policy Iteration" width=75% height=75% />

These algorithms have very similar value-of-state estimation cycle. The difference is that the *value iteration* algorithm iterates over **every** action while calculating the expected reward, and selects action with the maximal value. While the policy iteration calculates expected reward for the **single** action **specified by the policy**.

Sidenote: if $\mathbb{X}$ space is continuous, $x_j$ for both algorithms should be taken from a finite grid $\mathbb{X}_0$ over $\mathbb{X}$. For $x$ that does not belong to the grid the value of the function $V$ could be calculated via different approximations: closest neighbor-based, linear, etc.

In [ ]:
"""
DO NOT MODIFY
"""

class MDP:
    def __init__(self, env_name, is_slippery=False):
        self.env = gym.make(env_name, is_slippery=is_slippery)
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)
        self.gamma = 0.95
        self.nu = 0.0005
        
    def return_rewards(self):
        return 1.0 in self.rewards.values()

    def return_state_values(self):
        return tuple(self.values.items())

    def _model_transits_rewards(self, num_steps):
        """

        Description: step through the environment to model rewards and transits for all states. Also called "filling a buffer".

        Args:
            * num_steps - num steps to take through env. Should be sufficient to stochastically achieve goal of 1.
        """
        current_state = self.env.reset()

        print("Modeling rewards and transition probabilities ...")
    
        for i in tqdm(range(num_steps)):
            # sample random action
            action = self.env.action_space.sample()

            # take step
            new_state, reward, is_done, _ = self.env.step(action)
            
            # assign rewards for new state
            self.rewards[(current_state, action, new_state)] = reward

            # log transit from state to new state
            self.transits[(current_state, action)][new_state] += 1
            
            if is_done:
                current_state = self.env.reset()
            else:
                current_state = new_state

    def _get_action_value(self, current_state, action):
        """ 

        Description: Get the value of action

        Args:
            * State
            * Action

        Returns:
            * Value of current state

        """
        next_state_counts = self.transits[(current_state, action)]
        total_transits = sum(next_state_counts.values())
        action_value = 0.0
        
        for next_state, n_transits in next_state_counts.items():
            reward = self.rewards[(current_state, action, next_state)]
            transit_prob = (n_transits / total_transits)
            action_value += transit_prob * (reward + self.gamma * self.values[next_state])
        
        return action_value

    def _get_best_action(self, state):
        """ 

        Description: get best action for current state

        Args:
            * state

        Returns:
            * best action for current state

        """
        action_values = {}

        for action in range(self.env.action_space.n):
            action_value = self._get_action_value(state, action)
            action_values[action] = action_value

        best_action_value = max(action_values.values())
        best_action = max(action_values, key=action_values.get)
        
        return best_action

***

<h2 style="color:#A7BD3F;">Section 4: Problems</h2>

### <font color="blue">Problem 1.1 - Value Iteration</font>

Implement Value Iteration with the FrozenLake environment.

Guidance/hints:
* Read Chapter 4 from the class text
* Make sure you understand the value iteration algorithm
* Test taking steps through the FrozenLake environment by making your own script and executing it in new code cells
* Explore the MDP class above
* To complete the task, you'll need to make sure that your algorithm is properly calculating the state values. Call the `return_state_values` method to check the value of states. The value of the final state (aka the goal state) is always 0.

In [ ]:
"""
ADD YOUR CODE BETWEEN THE COMMENTS BELOW
"""
class ValueIteration(MDP):
    def __init__(self, env_name, is_slippery=False):
        super().__init__(env_name, is_slippery=is_slippery)
        
    def _value_iteration(self):
        """ 

        Description: Perform Value Iteration

        """
        #print("Performing value iteration ...")

        delta = 0
        iter_count = 1

        while True:
            ### YOUR SOLUTION BELOW
            
            ### YOUR SOLUTION ABOVE
            
            iter_count += 1

            if delta < self.nu:
                break                   
                
    def _run_episode(self, render=True):
        """

        Description: perform an episode on the environment

        Args
            * Render - render env to screen?

        """

        clear_output()
        episode_reward = 0.0
        current_state = self.env.reset()

        if render:
            self.env.render()
        
        while True:
            action = self._get_best_action(current_state)
            new_state, step_reward, is_done, _ = self.env.step(action)
            
            self.rewards[(current_state, action, new_state)] = step_reward
            self.transits[(current_state, action)][new_state] += 1
            
            episode_reward += step_reward
            
            if render:
                self.env.render()

            if is_done:
                self.env.reset()
                break
            
            current_state = new_state

        print(f"...Episode completed.")

        return episode_reward
        
    def run_simulation(self, num_steps = 1000, render=True):
        """ Run training simulation """
        try:
            ### YOUR SOLUTION BELOW
            
            ### YOUR SOLUTION ABOVE
            
            episode_reward = self._run_episode(render=render)

            if episode_reward > 0.85:
                print(f"Environment solved.")
            else:
                clear_output()
                print(f"Failed to solve environment.")

        except KeyboardInterrupt:
            print(f"...Cancelling...")
            
        except NotImplementedError:
            print(f"Your solution is incomplete")

### Run simulation

Execute the cell below. If your code is correct, you will see the environment render the agent taking steps. The final cell will be 'G'. 

**Note**: Running the simulation below with a large enough `num_steps` hyperparameter is vital to the value iteration algorithm working successfully. This is because the modeling process, which is stochastic (via `_model_transits_rewards`), requires sufficient iterations to reach the goal-state and acquire the reward in the environment, as well as to make enough transits to all possible states to make value iteration numerically stable.

In [ ]:
# DO NOT MODIFY

start_time = time.time()

agent1 = ValueIteration("FrozenLake-v0")
agent1.run_simulation(num_steps = 3000)

end_time = time.time()
print(f"Duration of execution: {end_time-start_time:.5f} seconds")

In [ ]:
#agent1.return_state_values()
#agent1.env.observation_space.n
agent1.values[6]

### State values?

In a 4x4 grid of FrozenLake, there are 16 states. Now let's look at their values:

In [ ]:
state_values = agent1.return_state_values()
state_values

In [ ]:
rewards = agent1.return_rewards()
rewards

### <font color="orange">Auto-grading</font>
Run this cell to track your answers and to save your answer for problem 1.1. Make sure you defined the necessary variable above to avoid a `NameError` below.

In [ ]:
### GRADING DO NOT MODIFY
from grading_utilities import AnswerTracker
asgn1_answers = AnswerTracker()
rewards_values = agent1.return_rewards()
asgn1_answers.record('problem_1-1', {'state_values': state_values, 'rewards': rewards_values})

### <font color="blue">Problem 1.2 - Policy Iteration</font>

Implement Policy Iteration on the FrozenLake environment.

In [ ]:
class PolicyIteration(MDP):
    def __init__(self, env_name, is_slippery=False):
        super().__init__(env_name, is_slippery=is_slippery)
        self.policy = collections.defaultdict(int)
        
    def return_policy(self):
        return tuple(self.policy.items())
        
    def _policy_iteration(self):
        """ 
        
        Description: Perform policy iteration. Consists of 2 parts: policy evaluation and policy improvement. See Sutton & Barto, RL: An Introduction, page 80.
        
        """

        #-------------------#
        # policy evaluation #
        #-------------------#

        delta = 0
        iter_count = 1

        while True:
            ### YOUR SOLUTION BELOW
            
            ### YOUR SOLUTION ABOVE
            
            iter_count += 1

            if delta < self.nu:
                break

        #--------------------#
        # policy improvement #
        #--------------------#

        policy_stable = np.zeros((self.env.observation_space.n), dtype=bool)

        ### YOUR SOLUTION BELOW
        
        ### YOUR SOLUTION ABOVE

        return policy_stable
    
    def _run_episode(self, render=True):
        """

        Description: runs an episode on the environment after policy iteration.

        Args:
            * Render - render env to screen?

        """
        clear_output()
        episode_reward = 0.0
        current_state = self.env.reset()

        if render:
            self.env.render()
        
        while True:
            action = self.policy[current_state]
            new_state, step_reward, is_done, _ = self.env.step(action)
            
            self.rewards[(current_state, action, new_state)] = step_reward
            self.transits[(current_state, action)][new_state] += 1
            
            episode_reward += step_reward
            
            if render:
                self.env.render()

            if is_done:
                self.env.reset()
                break
            
            current_state = new_state
            
        print(f"...Episode completed.")

        return episode_reward
        
    def run_simulation(self, num_steps = 2000, render=True):
        """ Run training simulation """
        try:
            self._model_transits_rewards(num_steps)

            while True:
                policy_stable = self._policy_iteration()

                if policy_stable.all() == True:
                    break

            episode_reward = self._run_episode(render=render)
            
            if episode_reward > 0.85:
                print(f"Environment solved.")
            else:
                clear_output()
                print(f"Failed to solve environment.")
        
        except KeyboardInterrupt:
            print(f"...Cancelling...")
            
        except NotImplementedError:
            print(f"Your solution is incomplete")

### Run simulation

As discussed above, beware of the num_steps hyperparam.

In [ ]:
start_time = time.time()

agent2 = PolicyIteration("FrozenLake-v0")
agent2.run_simulation(num_steps = 3000)

end_time = time.time()
print(f"Duration of execution: {end_time-start_time:.5f} seconds")

### <font color="orange">Auto-grading</font>
Run this cell to track your answers and to save your answer for problem 1.2. Make sure you defined the necessary variable above to avoid a `NameError` below.

In [ ]:
state_values = agent2.return_state_values()
policy_values = agent2.return_policy()

### GRADING DO NOT MODIFY
asgn1_answers.record('problem_1-2', {'state_values': state_values, 'policy_values': policy_values})

### <font color="blue">Problem 1.3 - FrozenLake8x8</font>

Let's try the 8x8 version of FrozenLake and compare which algorithm is faster.

In [ ]:
start_time = time.time()

# value iteration
agent3 = ValueIteration("FrozenLake8x8-v0")
agent3.run_simulation(render = False, num_steps = 50000)

end_time = time.time()
print(f"VI: Duration of execution: {end_time-start_time:.5f} seconds")

In [ ]:
%%timeit
agent3._value_iteration()

In [ ]:
start_time = time.time()

# policy iteration
agent4 = PolicyIteration("FrozenLake8x8-v0")
agent4.run_simulation(render = False, num_steps = 50000)

end_time = time.time()
print(f"PI: Duration of execution: {end_time-start_time:.5f} seconds")

In [ ]:
%%timeit
agent4._policy_iteration()

### Which algorithm is faster?

Which algorithm do you think is faster, on average, policy iteration (PI) or value iteration (VI)? The answer should be clear, if the algorithms are implemented correctly; try to find out, why is it so.

In [ ]:
"""
Change None to 'PI' or 'VI' below
"""
### YOUR ANSWER BELOW
problem_13_answer = ''
### YOUR ANSWER ABOVE

### <font color="orange">Auto-grading</font>
Run this cell to track your answers and to save your answer for problem 1.3. Make sure you defined the necessary variable above to avoid a `NameError` below.

In [ ]:
### GRADING DO NOT MODIFY
asgn1_answers.record('problem_1-3', problem_13_answer)

### <font color="orange">Auto-grading: Submit your answers</font>
Enter your first and last name in the cell below and then run it to save your answers for this lab to a JSON file. The file is saved to the same directory as this notebook. After the file is created, upload the JSON file to the assignment page on Canvas.

In [ ]:
asgn1_answers.print_answers()

In [ ]:
assignment_name = "asgn_1"
first_name = ""
last_name = ""

asgn1_answers.save_to_json(assignment_name, first_name, last_name)

## Questions?

Reach out to Ilya Osokin (@elijahmipt) on Telegram.

## Sources

***

<sup>[1]</sup> Ng, A. Stanford University, CS229 Notes: Reinforcement Learning and Control.

<sup>[2]</sup> Barnabás Póczos, Carnegie Mellon, Introduction To Machine Learning: Reinforcement Learning (Course).

<sup>[3]</sup> **Sutton, R. S., Barto, A. G. (2018 ). Reinforcement Learning: An Introduction. The MIT Press.** 

<sup>[4]</sup> OpenAI: Spinning Up. Retrieved from https://spinningup.openai.com/en/latest/spinningup/rl_intro.html